
# Summarize Youtube video lecture 
- Summarize Youtube's script by chapter creater configured. 
  - Create `markdown_note.md` with script and summary.
- Use [youtube-transcript-api](https://pypi.org/project/youtube-transcript-api/), [langchain](https://github.com/hwchase17/langchain), and [OpenAI](https://github.com/openai/openai-python) package. 


## Input variables

In [1]:
# Youtube video ID
youtube_video_id="esr-1s91_9A"

# Language of subscription 
language = "ko"

# LLM: Recommended parameters for my testing. 
max_token = 3000
model = "gpt-3.5-turbo"
chunk_size = 900
chunk_overlap = 50

# Officially no way to get chapter automatically, 
# so copy and paste the time stamp and chapter in description of Youtube video.

chapter_part_in_description = """
1:12 질서자유주의와 사회적 시장경제
3:49 실존철학의 메시지
6:32 독일 기본법(헌법)과 대한민국 헌법의 정신
18:58 우리는 어떻게 살아가고 있는가?
23:54 우리는 앞으로 어떻게 살아야 하는가?
28:56 피라미드형 계급구조 vs. 네트워크형 수평구조
32:56 의사결정은 반드시 합의를 거친다 (consensus)
45:54 대중의 지혜와 집단어리석음
50:10 ‘게르만 모형’의 공동결정법과 ‘일하는 방식’
54:55 아메리칸 드림에서 유러피언 드림으로
1:02:03 자기의식과 변증법적 역사발전
1:12:09 “우리는 다리를 놓으며 그 다리를 건너야 한다”
"""

In [2]:
# Officially no way to get chapter automatically, 
# so we need to parse the text in description and set up the dictionary 
# [ (time_in_sec, chapter_title) ]
import re 
pattern = r'(\d+(:\d+){1,2})\s(.+)'
matches = re.findall(pattern, chapter_part_in_description)

def time_to_seconds(time):
    parts = time.split(':')
    seconds = int(parts[-1])
    minutes = int(parts[-2]) if len(parts) > 1 else 0
    hours = int(parts[-3]) if len(parts) > 2 else 0
    return hours * 3600 + minutes * 60 + seconds

chapters = [(time_to_seconds(time), title.strip()) for time, _, title in matches]


# Build up note with chapter and script under each chapter 

In [3]:
from collections import deque
from youtube_transcript_api import YouTubeTranscriptApi

# Populate the script of YouTube video
data = YouTubeTranscriptApi.get_transcripts([youtube_video_id], languages=[language])
script_data = deque( data[0][youtube_video_id] )


# Put the script under each chatpter
# [ 
#   { 
#    "title": current_title,
#    "script": script_in_chapter
#    } ....
#  ]

script_by_chapter = []

script_in_chapter = ""
for i in range( len(chapters) ):
    current_time_in_sec, current_title = chapters[i]
    next_time_in_sec, next_title = chapters[i + 1] if i + 1 < len(chapters) else (None, None)

    if len(script_data) == 0:
        break

    s = script_data.popleft()
    end_time_of_script_in_sec = int( s['start'] + s['duration'] )

    if next_time_in_sec is not None:
        
        while end_time_of_script_in_sec < next_time_in_sec:
            script_in_chapter += s['text']
            script_in_chapter += " "
            s = script_data.popleft()
            end_time_of_script_in_sec = int( s['start'] + s['duration'] )

        chapter_data = { 
                        "title": current_title,
                        "script": script_in_chapter
                        }
        
        script_by_chapter.append(chapter_data)
        script_in_chapter = ""        

    else:
        script_in_chapter = ""

        while len(script_data) > 0 :
            script_in_chapter += s['text']
            script_in_chapter += " "
            s = script_data.popleft()
            end_time_of_script_in_sec = int( s['start'] + s['duration'] )

        chapter_data = { 
                        "title": current_title,
                        "script": script_in_chapter,
                        "summary" : ""
                        }
        
        script_by_chapter.append(chapter_data)



In [4]:

# Temporary save data into file 
import os 
import json 

with open( "temp_script_by_chapter.json", "w") as file:
    file.write( json.dumps(script_by_chapter, indent=2, ensure_ascii=False) )


# Write note by summarizing contents


In [5]:
# Setup OpenAI API key 
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import openai

def summarize_text_with_gpt3(text, max_token=3000, model="gpt-3.5-turbo", languages="ko"):

    prompt = f"""
        네가 대학생이고 아래 문장을 요약해서 노트를 만든다고 하자. 최대한 저자의 의도와 문장을 살려서 
        bulletin point를 붙여서 요약/정리해줘. 무언가 문장에 이상한 단어가 나오면 () 로 표시해줘.
        ----------
        {text}
        """
    # prompt = f"Summarize following text with bulletin points in Korean:\n{text}"
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_token
    )

    corrected_text = response.choices[0].message.content
    return corrected_text

In [7]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len
)


In [8]:
import time

# Summarize each chapter 
for c in script_by_chapter:
    texts = text_splitter.split_text(c["script"])

    title = c["title"]
    print( f"Chapter {title} is in-pro. ")
    
    summarized_text = ""
    for t in texts:
        time.sleep(0.05) # Avoid the bad request error. 
        partial_summary = summarize_text_with_gpt3(t, max_token = max_token, model = model)
        summarized_text += partial_summary
        print( ".", end="")

    c["summary"] = summarized_text
    print('\n')
    time.sleep(0.5) # Avoid the bad request error. 


Chapter 질서자유주의: 인간의 존엄성과 기능적 효율성이 조화된 질서 is in-pro. 
...

Chapter 사회적 시장경제 is in-pro. 
....

Chapter 칸트의 인간관 is in-pro. 
..

Chapter 인간이란 무엇인가? is in-pro. 
...

Chapter 인간의 존엄성과 자율성은 어디서 오는가? is in-pro. 
.............

Chapter 독일과 한국의 헌법정신 is in-pro. 
..

Chapter 역사의식과 시대정신 is in-pro. 
.

Chapter 정치인(공직자)의 책임은 무엇인가? is in-pro. 
...



## Publish markdown document

Find `markdown_note.md`. This is the summarized note for this Youtube video. 

In [9]:
with open( "script_by_chapter.json", "w") as file:
    file.write( json.dumps(script_by_chapter, indent=2, ensure_ascii=False) )


In [10]:
full_markdown_text = ""

for c in script_by_chapter:
    full_markdown_text += f"# {c['title']} \n\n"
    full_markdown_text += f"## Summary \n"
    full_markdown_text += f"{c['summary']} \n\n"
    full_markdown_text += f"## Script \n\n"
    full_markdown_text += f"{c['script']} \n"
    full_markdown_text += "\n\n"

In [11]:

# Write markdown document for note.
with open( "markdown_note.md", "w") as file:
    file.write(full_markdown_text)